In [1]:
%load_ext autoreload
%reload_ext tensorboard
%autoreload 2

In [2]:
from random import shuffle
from ssl import SSLSession
from tokenize import Whitespace
# from model_transformer import build_transformer
from dataset import BilingualDataset, causal_mask
from config import get_config, get_weights_file_path

import torchtext.datasets as datasets
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import LambdaLR

import warnings
from tqdm import tqdm
import os
from pathlib import Path

# Huggingface datasets and tokenizers
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

import torchmetrics
# Launch TensorBoard SSLSession
from torch.utils.tensorboard import SummaryWriter

/home/sn/anaconda3/envs/fastai2022/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [3]:
from config import get_config
import torch
import torch.optim as optim

from model_transformer import transformerModel, saveCallback #, PrintCallback #YOLOv3
from train import greedy_decode, get_ds
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")
from pytorch_lightning import LightningModule, Trainer

In [4]:
AVAIL_GPUS = min(1, torch.cuda.device_count())
AVAIL_GPUS

1

In [7]:
from pytorch_lightning.callbacks import ModelSummary
from pytorch_lightning.callbacks import LearningRateMonitor
lr_monitor = LearningRateMonitor(logging_interval='step')

num_examples = 2

cfg = get_config()
cfg['batch_size'] = 6
cfg['preload'] = None
cfg['num_epochs'] = 10

# Make sure the weights folder exists
Path(cfg['model_folder']).mkdir(parents=True, exist_ok=True)

train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(cfg)

#Tensorboard
writer = SummaryWriter(cfg['experiment_name'])

model = transformerModel(cfg, tokenizer_src, tokenizer_tgt, writer, num_examples)

trainer = Trainer(
    callbacks=[ModelSummary(max_depth=-1), lr_monitor], #saveCallback()], # PrintCallback()],
#     default_root_dir="/home/sn/ERAv1/checkpoints/",
    enable_checkpointing=True,
    # precision=16,
    devices=AVAIL_GPUS, 
    max_epochs=cfg['num_epochs'],
    # limit_train_batches = 10,
    limit_val_batches = num_examples,
    log_every_n_steps=500
) 

Max Length of source sentence: 309
Max Length of target sentence: 274


Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [8]:

trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)
#             ckpt_path="/ERAv1/session15_assignment/weights/s15Model.ckpt")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

    | Name                                                  | Type                    | Params
----------------------------------------------------------------------------------------------------
0   | model                                                 | Transformer             | 75.1 M
1   | model.encoder                                         | Encoder                 | 18.9 M
2   | model.encoder.layers                                  | ModuleList              | 18.9 M
3   | model.encoder.layers.0                                | EncoderBlock            | 3.1 M 
4   | model.encoder.layers.0.self_attention_block           | MultiHeadAttentionBlock | 1.0 M 
5   | model.encoder.layers.0.self_attention_block.w_q       | Linear                  | 262 K 
6   | model.encoder.layers.0.self_attention_block.w_k       | Linear                  | 262 K 
7   | model.encoder.layers.0.self_attention_block.w_v       | Linear                  | 262 K 

Sanity Checking: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
    SOURCE: Several wealthy and benevolent individuals in the county subscribed largely for the erection of a more convenient building in a better situation; new regulations were made; improvements in diet and clothing introduced; the funds of the school were intrusted to the management of a committee.
    TARGET: Molte benefiche e ricche persone della Contea riunirono una somma, che permise di ricostruire Lowood in modo più conveniente ed in posizione più salubre, furono fatti nuovi regolamenti, si migliorò il cibo e le vesti, e i fondi della scuola vennero affidati a un comitato.
 PREDICTED: Peggio congetture congetture congetture congetture congetture congetture congetture congetture congetture congetture congetture congetture congetture congetture congetture congetture congetture Peggio Peggio congetture Peggio congetture Peggio congetture Peggio Peggio congetture Peggio congetture Peggio congetture co

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
    SOURCE: It was Levin's face with his kind eyes looking mournfully from under his knit brows as he stood listening to her father and glancing at her and at Vronsky, and she felt so sorry for him that tears rose to her eyes.
    TARGET: Era il viso di Levin con le sopracciglia aggrottate e gli occhi buoni che guardavano di sotto in su, scoraggiati e tristi, mentre, in piedi, ascoltava suo padre e guardava lei e Vronskij. E provò tanta pena per lui che le vennero le lacrime agli occhi.
 PREDICTED: Era il viso di Levin , con gli occhi scintillanti , con le sopracciglia scintillanti , che , in particolare il padre , si era fermato e , guardando di lui , Vronskij , lo guardava con orrore e lacrime .
--------------------------------------------------------------------------------
    SOURCE: Karenin flushed, and pulling away his hand left the room.
    TARGET: Aleksej Aleksandrovic s’infiammò, e, svincolato b

Validation: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
    SOURCE: 'You didn't believe it, my darling?'
    TARGET: — Non ci credevi, piccolo mio?
 PREDICTED: — Non lo credi forse , mia cara ?
--------------------------------------------------------------------------------
    SOURCE: All the good counsels of my parents, my father’s tears and my mother’s entreaties, came now fresh into my mind; and my conscience, which was not yet come to the pitch of hardness to which it has since, reproached me with the contempt of advice, and the breach of my duty to God and my father.
    TARGET: Tutti i buoni consigli de’ miei genitori, le lagrime di mio padre, le preghiere di mia madre, mi si rinfrescarono nella memoria; e la mia coscienza che non era anche giunta a quell’eccesso d’indurimento, cui pervenne più tardi, mi rinfacciava il disprezzo de’ suggerimenti ricevuti e la violazione de’ miei doveri verso Dio e i miei genitori.
 PREDICTED: Tutti i miei timori , il mio

`Trainer.fit` stopped: `max_epochs=10` reached.


In [9]:
# start tensorboard
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
